In [2]:
# stdlib
import os

# syft absolute
import syft as sy

In [2]:
# node = sy.orchestra.launch(
#     name="test-domain-helm2",
#     dev_mode=True,
#     reset=True,
#     n_consumers=4,
#     create_producer=True,
# )
# client = node.login(email="info@openmined.org", password="changethis")

client = sy.login(
    url="http://localhost:8080", email="info@openmined.org", password="changethis"
)

Logged into <test: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [ ]:
job.get_result() -> block + return result

In [ ]:
job.wait().get(block=True)

In [ ]:
@syft_funciton
def my_func():
    res =domain.launch_job(very_long_function)
    return res

In [ ]:
job = my_func(blocking=False)

In [ ]:
job.wait(job_only=True)

In [ ]:
job.wait()

In [ ]:
def wait(job_only=False):
    

1. wait for the job to finish but not for the result to be ready
2. basic usecase: wait for the result


3. wait for the all the subjobs to finish

In [5]:
# [x.wait() for x in job.subjobs]

In [3]:
import random
import string

In [4]:
# import random
# import string

# def generate_random_line(length):
#     return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(length))

# def generate_large_file(file_path, num_lines, line_length):
#     with open(file_path, 'w') as file:
#         for _ in range(num_lines):
#             line = generate_random_line(line_length)
#             file.write(line + '\n')

# file_path = "large_file.txt"
# num_lines = 34359712  # Adjust the number of lines as needed
# line_length = 1000  # Adjust the line length as needed

# generate_large_file(file_path, num_lines, line_length)

In [5]:
# with open("somefile16gb.txt", 'rb') as input_file, open("somefile32gb.txt", 'ab') as output_file:
#     while True:
#         chunk = input_file.read(50 -random.randint(1, 10))
#         if not chunk:
#             break
#         output_file.write(chunk)
# #     while True:
# #         chunk = input_file.read(50 -random.randint(1, 10))
# #         if not chunk:
# #             break
# #         output_file.write(chunk)

In [4]:
lines_per_gb = int(2**30 / 1000)
gbs=32
with open(f'somefilerandom{gbs}gb.txt', 'wb') as f:
    for x in range(lines_per_gb*gbs):
        f.write(str.encode(''.join(random.choices(string.ascii_uppercase + string.digits, k=1000))) + b'\n')

In [7]:
from pathlib import Path

# syft absolute
from syft.client.client import SyftClient
from syft.store.blob_storage import BlobDeposit
from syft.types.blob_storage import CreateBlobStorageEntry
from syft.types.syft_object import SyftObject

In [8]:
def allocate_file(client: SyftClient, path: Path) -> BlobDeposit:
    create_blob_storage_entry = CreateBlobStorageEntry.from_path(path)
    return client.api.services.blob_storage.allocate(create_blob_storage_entry)

In [9]:
file = "somefile32gb.txt"

In [10]:
dep = allocate_file(client, Path(file))

In [11]:
dep

```python
class SeaweedFSBlobDeposit:
  id: str = 28d3991c2dbd4c2a8cb8e46f97f1196f

```

In [12]:
# from syft.client.api import APIRegistry

# api = APIRegistry.api_for(
#     node_uid=dep.syft_node_location,
#     user_verify_key=dep.syft_client_verify_key,
# )

# url = dep.urls[0]

# if api is not None:
#     blob_url = api.connection.to_blob_route(
#         url.url_path, host=url.host_or_ip
#     )
# else:
#     blob_url = url

# url = str(blob_url)

In [13]:
import requests

In [14]:
from syft import SyftError

In [15]:
from io import BytesIO

In [16]:
from typing import Generator

In [17]:
def _byte_chunks(bytes: BytesIO, size: int) -> Generator[bytes, None, None]:
    while True:
        try:
            yield bytes.read(size)
        except BlockingIOError:
            return

In [18]:
DEFAULT_CHUNK_SIZE = (1024**3 // 10)  # 100MB
DEFAULT_TIMEOUT = 5000  # in seconds

In [19]:
print(len(dep.urls))

321


In [20]:
from tqdm import tqdm

In [21]:
# byte_chunk[16384: 16384+10000]

In [22]:
# relative
with open(file, "rb") as data:
    from syft.client.api import APIRegistry

    api = APIRegistry.api_for(
        node_uid=dep.syft_node_location,
        user_verify_key=dep.syft_client_verify_key,
    )

    etags = []

    try:
        no_lines = 0
        for part_no, (byte_chunk, url) in tqdm(enumerate(
            zip(_byte_chunks(data, DEFAULT_CHUNK_SIZE), dep.urls),
            start=1,
        )):
            no_lines += byte_chunk.count(b"\n")
            if api is not None:
                blob_url = api.connection.to_blob_route(
                    url.url_path, host=url.host_or_ip
                )
            else:
                blob_url = url
                
            def data_generator(bytes_, chunk_size=819200):
                n=0
                while n*chunk_size <= len(bytes_):
                    chunk = bytes_[n*chunk_size:(n+1)*chunk_size]
                    n += 1
                    yield chunk
            print(part_no, no_lines)
            response = requests.put(
                url=str(blob_url), data=data_generator(byte_chunk), timeout=DEFAULT_TIMEOUT, stream=True
            )
            response.raise_for_status()
            etag = response.headers["ETag"]
            etags.append({"ETag": etag, "PartNumber": part_no})
    except requests.RequestException as e:
        print(e)




0it [00:00, ?it/s]

1 107265


1it [00:01,  1.97s/it]

2 214532


2it [00:03,  1.78s/it]

...

319it [11:18,  2.01s/it]

320 34325412


320it [11:20,  2.02s/it]

321 34359712


321it [11:21,  2.12s/it]


In [24]:
# len([x for x in data_generator(byte_chunk)])

In [23]:
(1024 ** 3) // 10

107374182

In [24]:
api.services.blob_storage.mark_write_complete(etags=etags, uid=dep.blob_storage_entry_id, no_lines=no_lines)

SyftSuccess: Successfully saved file.

In [ ]:
    # mark_write_complete_method = from_api_or_context(
    #     func_or_path="blob_storage.mark_write_complete",
    #     syft_node_location=self.syft_node_location,
    #     syft_client_verify_key=self.syft_client_verify_key,
    # )
    # return mark_write_complete_method(
    #     etags=etags, uid=self.blob_storage_entry_id, no_lines=no_lines
    # )

In [ ]:
with open("somefile.txt", "rb") as f:
    dep.write(f)


In [ ]:
x = BlobFile.upload_from_path("somefile.txt", client)

In [13]:
# # syft absolute
# from syft.store.blob_storage import BlobStorageClientConfig
# from syft.store.blob_storage import BlobStorageConfig
# from syft.store.blob_storage.seaweedfs import SeaweedFSClient
# from syft.store.blob_storage.seaweedfs import SeaweedFSClientConfig

# blob_config = BlobStorageConfig(
#     client_type=SeaweedFSClient,
#     client_config=SeaweedFSClientConfig(
#         host="http://0.0.0.0",
#         port="8333",
#         access_key="admin",
#         secret_key="admin",
#         bucket_name="test_bucket",
#         region="us-east-1",
#         # mount_port=4001
#     ),
# )
# node.python_node.init_blob_storage(blob_config)

In [14]:
from syft.types.blob_storage import BlobFile

In [8]:
x = BlobFile.upload_from_path("somefile.txt", client)

In [15]:
x = BlobFile.upload_from_path("somefile4gb.txt", client)

ConnectionError: Failed to fetch metadata. Response returned with code 500

In [9]:
x.read()

b'abcdef'

In [ ]:
"somefile.txt"

In [35]:
client.api.services.blob_storage.mount_azure(
    account_name="helmprojectstorage",
    container_name="helm",
    account_key=os.environ["HELM_STORAGE_ACCOUNT_KEY"],
    bucket_name="helmazurebucket",
)

SyftSuccess: Mounting Azure Successful!

In [42]:
files = client.api.services.blob_storage.get_files_from_bucket("helmazurebucket")

In [43]:
files

[syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile]

In [44]:
file = [f for f in files if f.file_name == "test.json"][0]

In [45]:
file

```python
class BlobFile:
  id: str = cb2ef738082c49418ed70eb05a193770
  file_name: str = "test.json"

```

In [46]:
file.read()

b'{\n"abc": "def"\n}'